# Social Graphs Project: Analyzing artists and songs

In [1]:
#pip install spotipy
#pip install bs4
# The one below requires to have git installed
#!pip install git+https://github.com/johnwmillr/LyricsGenius.git

  Cloning https://github.com/johnwmillr/LyricsGenius.git to c:\users\difto\appdata\local\temp\pip-req-build-asnsqzm3
  Resolved https://github.com/johnwmillr/LyricsGenius.git to commit bec02665b807941ca95e045be910e861789fc4a7
  Created wheel for lyricsgenius: filename=lyricsgenius-3.0.1-py3-none-any.whl size=45009 sha256=9a270dd399d8f72c34fbf03af1f09cafbcba31b2fad911943571e857068bfda9
  Stored in directory: C:\Users\difto\AppData\Local\Temp\pip-ephem-wheel-cache-3g246lfb\wheels\22\d8\54\6e236bc9517965a346a5b6cd223931b13f1eb6ba8676f1078c
Successfully built lyricsgenius


  Running command git clone -q https://github.com/johnwmillr/LyricsGenius.git 'C:\Users\difto\AppData\Local\Temp\pip-req-build-asnsqzm3'


In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import re
import requests
import json
import networkx as nx
from tqdm import tqdm
import json
import codecs
from requests.exceptions import ReadTimeout
import lyricsgenius

In [2]:
SPOTIFY_CLIENT_ID = "9a8ba8428ffc4b219099ad261a1366cf"
SPOTIFY_CLIENT_SECRET = "24b64e8cfee747889094e8a09da2b8f2"

# Spotify authentication (without user)
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

# Data collection

To start with, we want a list of artists (we chose the Wikipedia list of hip-hop artist).
Once we have it, we will, for every artist, find the list of all their songs, and all of the other artists they featured on a song.

In [3]:
# ------------- Getting a list of hip-hop artists from Wikipedia -----------------------------------------------------
# We will use the names for getting info from the Spotify API
# The wikipedia links can be useful should we need to get info from the artists' pages

# artists = {artist_name: [wiki_link, ...], ...}
artists = {}

# Parsing html content with beautifulsoup
soup = BeautifulSoup(requests.get("https://en.wikipedia.org/wiki/List_of_hip_hop_musicians").content, 'html.parser')
lists = soup.find_all(["div"], {"class": "div-col"})

for l in tqdm(lists):
    for match in re.findall(r"href=\"([\w\s=\/\(\).'%+!éè%$-]*)[\w\s=\"'\/\(\).\%+!éè$-]+>([.\w\s+'!éè$-]+)", str(l)):
        artists[match[1].lower()] = []
        artists[match[1].lower()].append(match[0])
        
print(len(artists), "artists were found on the Wikipedia page and added to the dictionary.")


# ------------- Getting the Spotify URI of every artist ---------------------------------------------------

temp = len(artists)
to_remove = []

for a in tqdm(artists):
    results = sp.search(q='artist:' + a, type='artist')
    # The item number 0 is the most popular result, so it is ikely to be the artist we're looking for
    try:
        artists[a].append(results['artists']['items'][0]['uri'])
    except:
        to_remove.append(a)

for a in to_remove:
    artists.pop(a)
    
print(temp-len(artists), "artists were not found with the Spotify API and were removed from the dictionary.")

100%|█████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 343.79it/s]


2830 artists were found on the Wikipedia page and added to the dictionary.


100%|██████████████████████████████████████████████████████████████████████████████| 2830/2830 [04:30<00:00, 10.48it/s]

50 artists were not found with the Spotify API and were removed from the dictionary.


In [4]:
# Saving the dictionary to a local text file

with codecs.open("C:/Users/difto/Desktop/artists.txt", 'w', "utf-8") as f:
    f.write(json.dumps(artists))

In [3]:
# Reloading the data from the file to make sure the cell runs on a clean version of it
with codecs.open("C:/Users/difto/Desktop/artists.txt", 'r', "utf-8") as f:
    data = json.load(f)

In [ ]:
# ------------- Getting all the songs from every artist, and every artist they featured on a song ----------------------------

# We will do the following data collection part in batches of, say, 100 artists because the Spotify API doesn't like to get
# too many requests and the code throws an error

artist_names = list(data.keys())
n = len(artist_names)
batch_indexes = [100 * i for i in range((n+100)//100)] + [n]

failedRequestIndexes = []


for i in range(len(batch_indexes)):
    print(i, ": Indexes from", batch_indexes[i], "to", batch_indexes[i+1])
    for j in tqdm(range(batch_indexes[i], batch_indexes[i+1])):
        feats = set()
        tracks_uris = set()
        # We give the artist's URI to the sp.artist_albums function
        
        try:
        
            for album in sp.artist_albums(data[artist_names[j]][1], limit=50)['items']:
                # We fetch the album data using the API
                r = sp.album_tracks(album['uri'])
                # We go through every song on the album
                for track_number in range(len(r['items'])):
                    # We might want to get also the song's name, in order to get the lyrics afterwards
                    
                    # Getting the URI
                    tracks_uris.add(r['items'][track_number]['uri'])
                    # Getting the artists featured
                    for item in r['items'][track_number]['artists']:
                        feats.add(item['name'].lower())
            data[artist_names[j]].append(list(feats))
            data[artist_names[j]].append(list(tracks_uris))
            
        except ReadTimeout:
            
            print("Spotify timed out for", j)
            failedRequestIndexes.append(j)
    
    # Saving the progress so far
    with codecs.open("C:/Users/difto/Desktop/Cours/02805 Social graphs and interactions/Project/Data/data"+str(i)+".txt", 'w', "utf-8") as f:
        f.write(json.dumps(artists))
    
    print(i, ": Done")

0 : Indexes from 0 to 100


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

In [ ]:
# If the cell above runs correctly, we end up with this format:
# data = {artist: [wikipedia_link, spotify_uri, [featured_artist, ...], [track_uri, ...]], ...}

In [4]:
for album in sp.artist_albums(data['lil mosey'][1], limit=50)['items']
    r = sp.album_tracks(album['uri'])
    # We go through every song on the album
    for track_number in range(len(r['items'])):
        # We might want to get also the song's name, in order to get the lyrics afterwards
        
        # Getting the URI
        tracks_uris.add(r['items'][track_number]['uri'])
        # Getting the artists featured
        for item in r['items'][track_number]['artists']:
            feats.add(item['name'].lower())

[{'album_group': 'album',
  'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5zctI4wO9XSKS8XwcnqEHk'},
    'href': 'https://api.spotify.com/v1/artists/5zctI4wO9XSKS8XwcnqEHk',
    'id': '5zctI4wO9XSKS8XwcnqEHk',
    'name': 'Lil Mosey',
    'type': 'artist',
    'uri': 'spotify:artist:5zctI4wO9XSKS8XwcnqEHk'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BF',
   'BG',
   'BH',
   'BI',
   'BJ',
   'BN',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BY',
   'BZ',
   'CA',
   'CD',
   'CG',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',
   'DE',
   'DJ',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GA',
   'GB',
   'GD',
   'GE',
   'GH',
   'GM',
   'GN',
   'GQ',
   'GR',
   'GT',
   'GW',
   'GY',
   'HK',
   'HN',
   

In [ ]:
r = sp.album_tracks(sp.artist_albums(data['lil mosey'][1], limit=50)['items'][0]['uri'])
r['items']

In [6]:
# Getting the lyrics of a given song from Genius

#GENIUS_CLIENT_ID = "gCDSyLMANvERLDi4GdVvKN6l7-QB1y2NHSkPr6QCUPVjKzEG_93CxN6Plj82nzWh"
GENIUS_ACCESS_TOKEN = "eAW6zYgc_Mf2dYp3xkouP8c8SuGjWg76mlVC95htgmJtOkD_OpKbwXK8aqfQMFRA"

LyricsGenius = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN)

try:
    song = LyricsGenius.search_song("lil mosey", "burberry headband")
except ReadTimeout or Timeout:
    pass
    
print(song.lyrics)

Searching for "lil mosey" by burberry headband...
Done.
Burberry Headband Lyrics[Intro]
Ayy, Royce, you did it right here
Skrrt, skrrt
Skrrt, skrrt
Ahhh, yah

[Chorus]
Pulled up in a Bimmer, I was flexin' (Skrrt, skrrt)
Poured a cup of Wock' 'cause I was stressin' (I was stressin')
After you, I'm fuckin' on your best friend (Lil' bitch)
She love me, she see Burberry my headband (On my headband)
Put on all my ice, now I feel so cool (I feel so cool)
You worry 'bout a nigga like a ho do (Like a ho do)
I never chase a bitch, I thought I told you (Never that)
I just chase the money like I'm 'posed to (Like I'm 'posed to)
[Verse 1]
Pull up, I was flexing out a Bimmer (Out a Bimmer)
Prolly why you mad 'cause my shit cleaner (Bitch)
Running to the bands like a competer (Like a huh?)
Fuck her 'til the mornin' then I leave her (Then I leave her)
Don't address me, you is a broke nigga, ayy
I don't wanna see you at my show nigga, ayy (At my show, nigga)
She wanna text me, she must throw throat, a

# Creating the graph

Now we can create the graph, following these rules:

1. Every node corresponds to one artist
1. An edge exists between two nodes if and only if the two corresponding artists are featured on a song (made by one of the two artists, i.e. for now we don't handle the case where, say, 3 artists are on a same song)

In [ ]:
# Creating the graph without node attributes for now

G = nx.Graph()

for artist in data:
    G.add_node(artist)
    for featured_artist in data[artist][2]:
        if featured_artist in data:
            G.add_edge(artist, featured_artist)